In [1]:
text = open('../makemore/gpt/input.txt', 'r').read()
len(text)

1115394

In [2]:
# print(text[:1000])

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [5]:
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [6]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [7]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8

In [9]:
# illustrate (spell it out with code) the examples a chunk of tokenized integers will make
# THIS CAN ALSO BE THOUGHT OF AS TIME DIMENSION
x = train_data[:block_size]
y = train_data[1:block_size+1]
print(x)
print(y)
print()

for t in range(block_size):
    context = x[ : t+1]
    target = y[t]
    print(f"when input is {context}: target is {target}")

tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([47, 56, 57, 58,  1, 15, 47, 58])

when input is tensor([18]): target is 47
when input is tensor([18, 47]): target is 56
when input is tensor([18, 47, 56]): target is 57
when input is tensor([18, 47, 56, 57]): target is 58
when input is tensor([18, 47, 56, 57, 58]): target is 1
when input is tensor([18, 47, 56, 57, 58,  1]): target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]): target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]): target is 58


In [10]:
torch.randint(9, (2,1))

tensor([[2],
        [1]])

In [11]:
# visualize a batch of data

torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-batch_size, (batch_size,))
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    return x,y

xb,yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("\n----------\n")

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, : t+1]
        target =  yb[b,     t]
        print(f"when input is {context}:      target is {target}")

inputs:
torch.Size([4, 8])
tensor([[ 1, 60, 39, 47, 50,  1, 63, 53],
        [46, 43, 39, 60, 43, 52,  1, 44],
        [ 1, 46, 43, 56, 43,  1, 63, 53],
        [61, 47, 50, 50,  1, 57, 39, 63]])
targets:
torch.Size([4, 8])
tensor([[60, 39, 47, 50,  1, 63, 53, 59],
        [43, 39, 60, 43, 52,  1, 44, 53],
        [46, 43, 56, 43,  1, 63, 53, 59],
        [47, 50, 50,  1, 57, 39, 63,  0]])

----------

when input is tensor([1]):      target is 60
when input is tensor([ 1, 60]):      target is 39
when input is tensor([ 1, 60, 39]):      target is 47
when input is tensor([ 1, 60, 39, 47]):      target is 50
when input is tensor([ 1, 60, 39, 47, 50]):      target is 1
when input is tensor([ 1, 60, 39, 47, 50,  1]):      target is 63
when input is tensor([ 1, 60, 39, 47, 50,  1, 63]):      target is 53
when input is tensor([ 1, 60, 39, 47, 50,  1, 63, 53]):      target is 59
when input is tensor([46]):      target is 43
when input is tensor([46, 43]):      target is 39
when input is tensor

## Bigram LM: simplest LM to start with

In [12]:
import torch
import torch.nn.functional as F
import torch.nn as nn
torch.manual_seed(1337)

In [19]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets = None):
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if targets == None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            
            # focus only on the last time step
            logits = logits[:,-1,:] # (B, C)
            
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        print("idx shape after concat::", idx.shape)
        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape) # (B*T,C)
print(loss)

print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.6413, grad_fn=<NllLossBackward0>)
idx shape after concat:: torch.Size([1, 101])

EGXxuxFFW:BkQW bnMoNi&zAyrONl?3XHzQmSBr&XxUnfeyI$aCSZRt:WI,tIGxKuGbOX;K-oRnM.VRFKORh.JCvATdTMv!ZPdiA


In [20]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [29]:
batch_size = 32
for steps in range(1000):
    xb, yb = get_batch('train')
    
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.511256694793701


In [32]:
print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

idx shape after concat:: torch.Size([1, 501])

ABupth.

Busiesed o I cesweven venourimemandel. aron, hald ke DUSouthyowourees g gerone ncr, m Gowiveay hord isa dwn:
Sore usoun fonest, arereane f-ery ollan
Touinghanor h isokinthat p ilutin, sthe pa soelf ath bear my madeay itheiman byouguomy blineny INur thauthrg msenlchas ave twoufo anaths
Wey t
AMIOhl d bond, tenowio h d d bo muthounothal our u sepustareamim I w, inoshyo nked itawhary slfor, sice d,
3--ENor halt r ge?
I HM:

GS Sooayonllirthime e:
Pr be es ivet
NI f nd omed!
I aknd beded al


----

## Some preliminaries work for Self-Attention

In [33]:
# consider a toy example

torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

### Easiest way for the token to communicate can be average out with all previous tokens

In [51]:
# version 1
# we want x[b,t] = mean_{i<=t} x[b,i]

xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #(t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [52]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [53]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

#### Efficiently doing the above interaction b/w timesteps incrementaly

In [47]:
# version 2

wei = torch.tril(torch.ones(T,T))
wei /= wei.sum(dim=1, keepdims=True)
wei # weights to do weighted aggregation

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [46]:
# wei.sum(dim=1)#, keepdims=True)

In [54]:
xbow2 = wei @ x # (T,T) @ (B,T,C) --> (B,T,C): Batched matrix mulitply

In [57]:
xbow2[0] # identical to what we acheive earlier with 2 for-loops

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [56]:
torch.allclose(xbow, xbow2)

True

In [80]:
# version 3

tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)

In [81]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [82]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [84]:
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [104]:
# torch.arange(T)

In [102]:
# version 4: self-attention

torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# single head performing self-attention
head_size = 16
query = nn.Linear(C, head_size, bias=False)
key = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, head_size)
q = query(x) # (B, T, head_size)

wei = q @ k.transpose(-2,-1) # (B,T,16) @ (B,16,T) --> (B,T,T)


tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x) # (B, T, head_size)
out = wei @ v # (B,T,T) @ (B,T,head_size) --> (B,T,head_size)

print(out.shape)

torch.Size([4, 8, 16])


In [103]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5877, 0.4123, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4457, 0.2810, 0.2733, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2220, 0.7496, 0.0175, 0.0109, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0379, 0.0124, 0.0412, 0.0630, 0.8454, 0.0000, 0.0000, 0.0000],
        [0.5497, 0.2187, 0.0185, 0.0239, 0.1831, 0.0062, 0.0000, 0.0000],
        [0.2576, 0.0830, 0.0946, 0.0241, 0.1273, 0.3627, 0.0507, 0.0000],
        [0.0499, 0.1052, 0.0302, 0.0281, 0.1980, 0.2657, 0.1755, 0.1474]],
       grad_fn=<SelectBackward0>)

## Understanding how concat with diff dim values works

In [133]:
a = torch.randn(4,2,1)
b = torch.randn(4,2,1)
print(a, a.ndim)
print(b, b.ndim)

tensor([[[-0.8204],
         [-0.5869]],

        [[-0.3027],
         [ 1.4529]],

        [[ 1.8694],
         [ 1.0485]],

        [[-0.2940],
         [-0.4703]]]) 3
tensor([[[ 0.8907],
         [-0.8033]],

        [[-0.2139],
         [ 1.4153]],

        [[-1.2557],
         [ 0.4283]],

        [[ 1.1396],
         [ 1.6915]]]) 3


In [142]:
torch.cat([a,b], dim=-2).shape, torch.cat([a,b], dim=-1)

(torch.Size([4, 4, 1]),
 tensor([[[-0.8204,  0.8907],
          [-0.5869, -0.8033]],
 
         [[-0.3027, -0.2139],
          [ 1.4529,  1.4153]],
 
         [[ 1.8694, -1.2557],
          [ 1.0485,  0.4283]],
 
         [[-0.2940,  1.1396],
          [-0.4703,  1.6915]]]))